# 加载1.8B Chat Int4模型

In [204]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4')

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

2023-12-31 15:06:46,295 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
Try importing flash-attention for faster inference...


# 提示词工程

In [205]:
# 城市数据
with open('city.txt','r',encoding='utf-8') as fp:
    city_list=fp.readlines()
    city_list=[line.strip().split(' ')[1] for line in city_list]

In [206]:
Q='1月2号青岛天气预报'

prompt_template='''
分析问题中出现过的城市和日期

交互格式如下：

问题: 用户的原始问题，里面可能包含了城市和日期信息
回答: 提炼问题中出现的城市名称和具体日期，按如下格式整理返回:
    city: 值是城市的名字
    date: 值是具体日期

现在开始。

问题: %s
回答: 
'''

prompt=prompt_template%(Q,)
#print(prompt)

resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)

city: 青岛
date: 1月2号


# 生成SFT微调数据

Qwen的SFT数据格式要求:
```
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好"
      },
      {
        "from": "assistant",
        "value": "我是一个语言模型，我叫通义千问。"
      }
    ]
  }
]
```

In [207]:
import random
import json
import time 

train_data=[]

Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    Q=Q_list[random.randint(0,len(Q_list)-1)]
    city=city_list[random.randint(0,len(city_list)-1)]
    year=random.randint(1990,2025)
    month=random.randint(1,12)
    day=random.randint(1,28)
    time_str='{}-{}-{}'.format(year,month,day)
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))
    
    Q=Q[0].format(city=city,year=year,month=month,day=day) # 问题
    A='city:%s\ndate:%s'%(city,date_field) # 回答
    example={
        'id': 'identity_{}'.format(i),
        'conversations':[
            {
                'from':'user',
                'value':prompt_template%(Q,),
            },
            {
                'from':'assistant',
                'value':A,
            },
        ]
    }
    #print(example)
    train_data.append(example)

with open('train.txt','w',encoding='utf-8') as fp:
    fp.write(json.dumps(train_data))
print('样本数量:',len(train_data))

样本数量: 1000


# 微调模型，生成到output_qwen

bash finetune/finetune_qlora_single_gpu.sh  -m /root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4 -d /root/Qwen/train.txt

# 加载SFT后的模型

In [208]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    'output_qwen', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

Try importing flash-attention for faster inference...


In [212]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？']
for Q in Q_list:
    prompt=prompt_template%(Q,)
    resp,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,resp))

Q:2020年4月16号三亚下雨么？
A:city:三亚
date:2020-04-16

Q:青岛3-15号天气预报
A:city:青岛
date:03-15

Q:5月6号下雪么，城市是威海
A:city:威海
date:05-06

Q:青岛2023年12月30号有雾霾么?
A:city:青岛
date:2023-12-30

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:city:北京市
date:06-01



In [213]:
prompt='青岛海边钓鱼需要特别注意什么？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)

青岛海边钓鱼需要注意以下几点：
1、选择合适的地点：青岛海边的鱼塘一般分为几个区域，每个区域都有自己的特点和优势。比如北部海域水质较好，适合鱼类生长；南部海域水质较差，适合鱼类避风。
2、选择合适的季节：青岛海边钓鱼的最佳季节是春季和秋季，这两个季节温度适宜，而且海风较大，有利于鱼类的活动。
3、选择合适的工具：青岛海边钓鱼常用的工具包括钓竿、饵料、鱼线、鱼钩等。其中钓竿的选择要根据个人的技术水平来定，饵料的选择要根据当地的海洋环境来定。
4、注意安全：青岛海边钓鱼时要注意安全，不要在危险的地方钓鱼，也不要随意丢弃垃圾。
